In [98]:
%run utils.py

In [112]:
test = """\
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""

valid = """\
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""

invalid = """
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""

# count those that have all required fields. Treat cid as optional.
key_set = {'byr', 'cid', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
opt_key = {'cid'}
regex_input = "(\w+):([a-zA-Z0-9#]+)"
check_fun = {
    'byr': lambda x: 1 if len(x)==4 and int(x) in range(1920,2002+1) else 0,
    'iyr': lambda x: 1 if len(x)==4 and int(x) in range(2010,2020+1) else 0,
    'ecl': lambda x: 1 if x in ['amb','blu','brn','gry','grn','hzl','oth'] else 0,
    'eyr': lambda x: 1 if len(x)==4 and int(x) in range(2020,2030+1) else 0,
    'hcl': lambda x: 1 if len([n for (n) in re.findall("(^#[a-fA-F0-9#]+)",x) if len(n)==6+1]) == 1 else 0,
    'hgt': lambda x: 1 if len([int(h) for (h,u) in re.findall("(\d+)(cm|in)",x) if (u=='cm' and int(h) in range(150,193+1)) or (u=='in' and int(h) in range(59,76+1))]) == 1 else 0,
    'pid': lambda x: 1 if len([n for (n) in re.findall("(\d+)",x) if len(n)==9]) == 1 else 0,
    'cid': lambda x: 1
}

def test_reg(text:str):
    for s in text.split('\n\n'):
        yield re.findall(regex_input,s)

list(test_reg(test))

[[('ecl', 'gry'),
  ('pid', '860033327'),
  ('eyr', '2020'),
  ('hcl', '#fffffd'),
  ('byr', '1937'),
  ('iyr', '2017'),
  ('cid', '147'),
  ('hgt', '183cm')],
 [('iyr', '2013'),
  ('ecl', 'amb'),
  ('cid', '350'),
  ('eyr', '2023'),
  ('pid', '028048884'),
  ('hcl', '#cfa07d'),
  ('byr', '1929')],
 [('hcl', '#ae17e1'),
  ('iyr', '2013'),
  ('eyr', '2024'),
  ('ecl', 'brn'),
  ('pid', '760753108'),
  ('byr', '1931'),
  ('hgt', '179cm')],
 [('hcl', '#cfa07d'),
  ('eyr', '2025'),
  ('pid', '166559648'),
  ('iyr', '2011'),
  ('ecl', 'brn'),
  ('hgt', '59in')]]

In [113]:
def tok(iter_in):
    def mk_dict(kv_tuple:list) -> dict:
        return {k:v for (k,v) in kv_tuple}
    
    def valid_key_set(s:set) -> int:
        return 1 if s >= key_set-opt_key else 0
    
    def valid_key_value(d:dict) -> int:
        return 1 if sum([check_fun[k](v) for k,v in d.items() ]) == len(set(d.keys())) else 0

    accum = []
    for line in iter_in:
        l = re.findall(regex_input,line)
        #print(l)
        if len(l) :
            accum += l
        else :
            yield valid_key_set(set(mk_dict(accum))) and valid_key_value(mk_dict(accum))
            accum = []
    # get last accumulated
    yield valid_key_set(set(mk_dict(accum))) and valid_key_value(mk_dict(accum))



In [114]:
sum(list(tok(test.splitlines())))

2

In [119]:
sum(list(tok(valid.splitlines()))) == len(valid.split('\n\n'))

True

In [120]:
sum(list(tok(invalid.splitlines()))) == 0

True

In [118]:
sum(list(tok(open("input/input4.txt"))))

175

In [59]:
fun = lambda x: 1 if len([int(h) for (h,u) in re.findall("(\d+)(cm|in)",x) if (u=='cm' and int(h) in range(150,193+1)) or (u=='in' and int(h) in range(59,76+1))]) == 1 else 0
list(map(fun,'150cm, 59in, 193cm 76in 179cm 202cm 145cm 67in'.split()))


[1, 1, 1, 1, 1, 0, 0, 1]

In [86]:
fun = lambda x: 1 if len([n for (n) in re.findall("(\d+)",x) if len(n)==9]) == 1 else 0
list(map(fun,'0001230 1234567890 760753108'.split()))

[0, 0, 1]

In [97]:
re.findall("(^#[a-fA-F0-9#]+)",'#abcdef')
fun = lambda x: 1 if len([n for (n) in re.findall("(^#[a-fA-F0-9#]+)",x) if len(n)==6+1]) == 1 else 0
list(map(fun,'#abcdef abcdef #11eeda'.split()))

[1, 0, 1]